# Bad code below

In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import csv
from math import sqrt
from copy import deepcopy

In [3]:
input_data = []
output_data = []
with open('eng_de.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        input_data.append(row[0].strip().split())
        output_data.append(row[1].strip().split())

In [13]:
all_words = np.unique([word for row in input_data for word in row]+[word for row in output_data for word in row])
word_dict = {all_words[i]:i for i in range(len(all_words))}
word_dict

{'apfel': 0,
 'apple': 1,
 'beer': 2,
 'bier': 3,
 'book': 4,
 'bread': 5,
 'brot': 6,
 'buch': 7,
 'can': 8,
 'drink': 9,
 'eat': 10,
 'essen': 11,
 'i': 12,
 'ich': 13,
 'konnen': 14,
 'lesen': 15,
 'mochten': 16,
 'newspaper': 17,
 'read': 18,
 'to': 19,
 'trinken': 20,
 'want': 21,
 'wasser': 22,
 'water': 23,
 'we': 24,
 'wir': 25,
 'zeitung': 26}

In [14]:
one_hot_inputs = np.zeros((len(input_data),10,32))
for sentence_ind in range(len(input_data)):
    for word_ind in range(len(input_data[sentence_ind])):
        one_hot_inputs[sentence_ind][word_ind][word_dict[input_data[sentence_ind][word_ind]]] = 1


one_hot_outputs = np.zeros((len(output_data),10,32))
for sentence_ind in range(len(output_data)):
    for word_ind in range(len(output_data[sentence_ind])):
        one_hot_outputs[sentence_ind][word_ind][word_dict[output_data[sentence_ind][word_ind]]] = 1

In [15]:
class PositionalEncoding(keras.layers.Layer):
    def __init__(self,d_model,max_seq_len):
        super(PositionalEncoding,self).__init__()
        self.encodings = np.array([pos/np.power(10_000,(np.arange(d_model)//2)/d_model) for pos in range(max_seq_len)])
        self.encodings[:,0::2] = np.sin(self.encodings[:,0::2])
        self.encodings[:,1::2] = np.cos(self.encodings[:,1::2])
        
        
    def call(self,inputs):
        if inputs.shape[1]!=None:
            return inputs+self.encodings[:inputs.shape[1]]
        else:
            return inputs#this is stupid
                             

class AttentionHead(keras.layers.Layer):
    def __init__(self,d_model,d_k):
        super(AttentionHead,self).__init__()
        self.w_q = tf.Variable(np.random.uniform(-1.0,1.0,size=(d_model,d_k)),dtype=tf.float32)
        self.w_k = tf.Variable(np.random.uniform(-1.0,1.0,size=(d_model,d_k)),dtype=tf.float32)
        self.w_v = tf.Variable(np.random.uniform(-1.0,1.0,size=(d_model,d_k)),dtype=tf.float32)
        self.d_model = d_model
        self.d_k = d_k
    
    def call(self,Q,K,V):
        queries = tf.matmul(Q,self.w_q)
        keys = tf.matmul(K,self.w_k)
        values = tf.matmul(V,self.w_v)
        scores = tf.nn.softmax(tf.matmul(queries,tf.transpose(keys,perm=[0,2,1]))/sqrt(self.d_k))
        return tf.matmul(scores,values)

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self,d_model,n_heads):
        super(MultiHeadAttention,self).__init__()
        assert d_model%n_heads==0
        
        d_k = d_model//n_heads
        self.heads = [AttentionHead(d_model,d_k) for _ in range(n_heads)]
        self.w_o = tf.Variable(np.random.uniform(-1.0,1.0,size=(d_model,d_model)),dtype=tf.float32)
    
    def call(self,Q,K,V):
        heads_outputs = tf.concat([head(Q,K,V) for head in self.heads],-1)
        return tf.matmul(heads_outputs,self.w_o)

class EncoderBlock(keras.layers.Layer):
    def __init__(self,d_model,n_heads):
        super(EncoderBlock,self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.attention = MultiHeadAttention(d_model,n_heads)
        self.norm_layer_1 = keras.layers.LayerNormalization()
        self.ff_layer = keras.layers.Dense(d_model,activation='relu')
        self.norm_layer_2 = keras.layers.LayerNormalization()
    
    def call(self,inputs):
        x = inputs + self.attention(inputs,inputs,inputs)
        x = self.norm_layer_1(x)
        x = x + self.ff_layer(x)
        return self.norm_layer_2(x)

class DecoderBlock(keras.layers.Layer):
    def __init__(self,d_model,n_heads):
        super(DecoderBlock,self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        
        self.attention_1 = MultiHeadAttention(d_model,n_heads)
        self.norm_layer_1 = keras.layers.LayerNormalization()
        self.attention_2 = MultiHeadAttention(d_model,n_heads)
        self.norm_layer_2 = keras.layers.LayerNormalization()
        self.ff_layer = keras.layers.Dense(d_model,activation='relu')
        self.norm_layer_3 = keras.layers.LayerNormalization()
    
    def call(self,prev_tokens,encoder_tokens):
        x = prev_tokens + self.attention_1(prev_tokens,prev_tokens,prev_tokens)
        x = self.norm_layer_1(x)
        x = x + self.attention_2(prev_tokens,prev_tokens,encoder_tokens)
        x = self.norm_layer_2(x)
        x = x + self.ff_layer(x)
        return self.norm_layer_3(x)


    
def make_transformer(n_encoders=3,n_decoders=3,n_heads=4,token_size=16,max_tokens=10):
    encoder_input = keras.Input(shape=(None,token_size),dtype='float32',name='encoder_input')
    encoder_output = PositionalEncoding(token_size,max_tokens)(encoder_input)
    for encoder in range(n_encoders):
        encoder_output = EncoderBlock(token_size,n_heads)(encoder_output)
    
    decoder_input = keras.Input(shape=(None,token_size),dtype='float32',name='decoder_input')
    decoder_output = PositionalEncoding(token_size,max_tokens)(decoder_input)
    for decoder in range(n_decoders):
        decoder_output = DecoderBlock(token_size,n_heads)(decoder_output,encoder_output)
    
    transformer_model = keras.Model(inputs = [encoder_input,decoder_input],outputs=decoder_output)
    transformer_model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )
    return transformer_model
        

In [8]:
def train_generator(inputs,outputs,mask_val=-1.0):
    while(True):
        batch = []
        for seq_ind in range(len(inputs)):
            batch_encoder_input = np.array([inputs[seq_ind] for _ in range(len(outputs[seq_ind]))])
            batch_decoder_input = np.array([outputs[seq_ind] for _ in range(len(outputs[seq_ind]))])
            batch_output = deepcopy(batch_decoder_input)
            for mask_ind in range(len(outputs[seq_ind])):
                batch_decoder_input[mask_ind][mask_ind+1:] = mask_val
            yield {'encoder_input':batch_encoder_input,'decoder_input':batch_decoder_input},batch_output

In [16]:
N_EPOCHS = 500
EPOCH_STEPS = 50

In [17]:
transformer = make_transformer(token_size=32)

#use this to plot model topology
# keras.utils.plot_model(transformer,show_shapes=False)

#use to test
# outie = transformer({'encoder_input':one_hot_inputs,'decoder_input':one_hot_outputs})

In [18]:
hist = transformer.fit(x=train_generator(one_hot_inputs,one_hot_outputs),verbose=1,epochs=N_EPOCHS,steps_per_epoch=EPOCH_STEPS)

Epoch 1/500
50/50 [==============================] - 13s 48ms/step - loss: 2.7483 - accuracy: 0.3160
Epoch 2/500
50/50 [==============================] - 2s 47ms/step - loss: 2.2659 - accuracy: 0.3880
Epoch 3/500
50/50 [==============================] - 2s 48ms/step - loss: 1.8484 - accuracy: 0.2460
Epoch 4/500
50/50 [==============================] - 2s 47ms/step - loss: 1.6382 - accuracy: 0.2900
Epoch 5/500
50/50 [==============================] - 2s 48ms/step - loss: 1.4924 - accuracy: 0.2700
Epoch 6/500
50/50 [==============================] - 2s 48ms/step - loss: 1.3114 - accuracy: 0.2180
Epoch 7/500
50/50 [==============================] - 2s 48ms/step - loss: 1.2771 - accuracy: 0.2600
Epoch 8/500
50/50 [==============================] - 2s 49ms/step - loss: 1.2592 - accuracy: 0.2160
Epoch 9/500
50/50 [==============================] - 2s 48ms/step - loss: 1.2184 - accuracy: 0.2100
Epoch 10/500
50/50 [==============================] - 2s 49ms/step - loss: 1.2282 - accuracy: 0.280

50/50 [==============================] - 2s 49ms/step - loss: 1.5070 - accuracy: 0.0160
Epoch 83/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5088 - accuracy: 0.0160
Epoch 84/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5090 - accuracy: 0.0180
Epoch 85/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5023 - accuracy: 0.0160
Epoch 86/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5123 - accuracy: 0.0160
Epoch 87/500
50/50 [==============================] - 2s 50ms/step - loss: 1.5028 - accuracy: 0.0180
Epoch 88/500
50/50 [==============================] - 2s 50ms/step - loss: 1.5077 - accuracy: 0.0160
Epoch 89/500
50/50 [==============================] - 2s 50ms/step - loss: 1.5129 - accuracy: 0.0160
Epoch 90/500
50/50 [==============================] - 2s 48ms/step - loss: 1.4960 - accuracy: 0.0180
Epoch 91/500
50/50 [==============================] - 2s 50ms/step - loss: 1.5140 - accuracy: 0.0160
Epo

50/50 [==============================] - 2s 49ms/step - loss: 1.4915 - accuracy: 0.0180
Epoch 163/500
50/50 [==============================] - 2s 47ms/step - loss: 1.5089 - accuracy: 0.0160
Epoch 164/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5027 - accuracy: 0.0160
Epoch 165/500
50/50 [==============================] - 2s 49ms/step - loss: 1.4953 - accuracy: 0.0180
Epoch 166/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5088 - accuracy: 0.0160
Epoch 167/500
50/50 [==============================] - 2s 48ms/step - loss: 1.4969 - accuracy: 0.0160
Epoch 168/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5011 - accuracy: 0.0180
Epoch 169/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5068 - accuracy: 0.0160
Epoch 170/500
50/50 [==============================] - 2s 49ms/step - loss: 1.4969 - accuracy: 0.0160
Epoch 171/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5030 - accuracy: 0

50/50 [==============================] - 2s 48ms/step - loss: 1.4966 - accuracy: 0.0160
Epoch 243/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5028 - accuracy: 0.0180
Epoch 244/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5008 - accuracy: 0.0160
Epoch 245/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5024 - accuracy: 0.0160
Epoch 246/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5028 - accuracy: 0.0180
Epoch 247/500
50/50 [==============================] - 2s 50ms/step - loss: 1.4970 - accuracy: 0.0160
Epoch 248/500
50/50 [==============================] - 2s 49ms/step - loss: 1.5063 - accuracy: 0.0160
Epoch 249/500
50/50 [==============================] - 2s 49ms/step - loss: 1.4970 - accuracy: 0.0180
Epoch 250/500
50/50 [==============================] - 2s 48ms/step - loss: 1.5028 - accuracy: 0.0160
Epoch 251/500
50/50 [==============================] - 2s 50ms/step - loss: 1.5063 - accuracy: 0

50/50 [==============================] - 3s 51ms/step - loss: 0.5819 - accuracy: 0.0160
Epoch 323/500
50/50 [==============================] - 2s 50ms/step - loss: 0.5816 - accuracy: 0.0160
Epoch 324/500
50/50 [==============================] - 2s 49ms/step - loss: 0.5742 - accuracy: 0.0180
Epoch 325/500
50/50 [==============================] - 3s 50ms/step - loss: 0.5861 - accuracy: 0.0160
Epoch 326/500
50/50 [==============================] - 3s 51ms/step - loss: 0.5792 - accuracy: 0.0160
Epoch 327/500
50/50 [==============================] - 2s 50ms/step - loss: 0.5780 - accuracy: 0.0180
Epoch 328/500
50/50 [==============================] - 3s 50ms/step - loss: 0.5831 - accuracy: 0.0160
Epoch 329/500
50/50 [==============================] - 3s 51ms/step - loss: 0.5765 - accuracy: 0.0160
Epoch 330/500
50/50 [==============================] - 2s 49ms/step - loss: 0.5838 - accuracy: 0.0180
Epoch 331/500
50/50 [==============================] - 2s 49ms/step - loss: 0.5812 - accuracy: 0

50/50 [==============================] - 2s 48ms/step - loss: 0.5690 - accuracy: 0.0180
Epoch 403/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5727 - accuracy: 0.0160
Epoch 404/500
50/50 [==============================] - 2s 49ms/step - loss: 0.5641 - accuracy: 0.0160
Epoch 405/500
50/50 [==============================] - 2s 49ms/step - loss: 0.5802 - accuracy: 0.0180
Epoch 406/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5797 - accuracy: 0.0160
Epoch 407/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5812 - accuracy: 0.0160
Epoch 408/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5836 - accuracy: 0.0180
Epoch 409/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5746 - accuracy: 0.0160
Epoch 410/500
50/50 [==============================] - 2s 48ms/step - loss: 0.5846 - accuracy: 0.0160
Epoch 411/500
50/50 [==============================] - 2s 50ms/step - loss: 0.5777 - accuracy: 0

50/50 [==============================] - 4s 82ms/step - loss: 0.6044 - accuracy: 0.0160
Epoch 483/500
50/50 [==============================] - 4s 82ms/step - loss: 0.5967 - accuracy: 0.0180
Epoch 484/500
50/50 [==============================] - 4s 84ms/step - loss: 0.5986 - accuracy: 0.0160
Epoch 485/500
50/50 [==============================] - 4s 83ms/step - loss: 0.6044 - accuracy: 0.0160
Epoch 486/500
50/50 [==============================] - 4s 83ms/step - loss: 0.5909 - accuracy: 0.0180
Epoch 487/500
50/50 [==============================] - 4s 81ms/step - loss: 0.6044 - accuracy: 0.0160
Epoch 488/500
50/50 [==============================] - 4s 80ms/step - loss: 0.6005 - accuracy: 0.0160
Epoch 489/500
50/50 [==============================] - 4s 82ms/step - loss: 0.5948 - accuracy: 0.0180
Epoch 490/500
50/50 [==============================] - 4s 80ms/step - loss: 0.6044 - accuracy: 0.0160
Epoch 491/500
50/50 [==============================] - 4s 80ms/step - loss: 0.5948 - accuracy: 0

In [276]:
gen = train_generator(one_hot_inputs,one_hot_outputs)
test_inputs, test_outputs = next(gen)
pred = transformer(test_inputs)
pred

<tf.Tensor: shape=(10, 10, 32), dtype=float32, numpy=
array([[[ 0.08211152, -0.20555604, -0.16429874, ..., -0.66962045,
         -0.5916419 , -0.5481715 ],
        [ 0.08211157, -0.20555602, -0.16429877, ..., -0.6696204 ,
         -0.59164184, -0.5481714 ],
        [ 0.08211158, -0.20555599, -0.16429877, ..., -0.66962045,
         -0.59164184, -0.5481714 ],
        ...,
        [ 0.08211157, -0.20555602, -0.16429877, ..., -0.6696204 ,
         -0.59164184, -0.5481714 ],
        [ 0.08211157, -0.20555602, -0.16429877, ..., -0.6696204 ,
         -0.59164184, -0.5481714 ],
        [ 0.08211158, -0.20555599, -0.16429877, ..., -0.66962034,
         -0.59164184, -0.54817134]],

       [[ 0.08211109, -0.20555617, -0.16429842, ..., -0.6696202 ,
         -0.5916417 , -0.5481709 ],
        [ 0.08211105, -0.20555621, -0.16429844, ..., -0.66962034,
         -0.5916417 , -0.54817086],
        [ 0.08211105, -0.20555627, -0.1642986 , ..., -0.6696203 ,
         -0.59164166, -0.5481708 ],
        ...,


In [277]:
pred[0]

<tf.Tensor: shape=(10, 32), dtype=float32, numpy=
array([[ 0.08211152, -0.20555604, -0.16429874, -1.5145501 , -0.37675244,
        -0.81116295,  3.4811935 , -0.08330107, -0.53607965, -0.7392224 ,
        -0.3941177 , -0.88468444, -0.55713075, -0.59213126, -0.7105564 ,
        -0.46688604, -0.80185425, -0.20627993, -0.3647418 ,  0.06609145,
        -0.08202042, -0.65933204, -0.81193537, -0.14005584, -0.36213696,
        -0.15784243, -0.82751   , -0.28333318, -0.8281327 , -0.66962045,
        -0.5916419 , -0.5481715 ],
       [ 0.08211157, -0.20555602, -0.16429877, -1.5145501 , -0.3767524 ,
        -0.81116307,  3.4811935 , -0.08330107, -0.5360796 , -0.7392223 ,
        -0.39411765, -0.88468456, -0.5571307 , -0.5921311 , -0.7105564 ,
        -0.46688598, -0.8018542 , -0.20627995, -0.3647417 ,  0.06609151,
        -0.08202031, -0.6593319 , -0.8119353 , -0.14005578, -0.36213687,
        -0.15784234, -0.82751   , -0.2833331 , -0.8281327 , -0.6696204 ,
        -0.59164184, -0.5481714 ],
    